Description: Jupyter notebook for carrying out different MLE stragies on simulated datasets

In [1]:
# Imports
import os, sys, types
import pathlib

In [2]:
# Add paths
import itertools
import math
import sympy
import numpy as np
import scipy.linalg
import scipy.stats
import scipy.optimize
import scipy.fftpack

import tensorflow  as  tf
tf.compat.v1.enable_eager_execution(config=None, device_policy=None, execution_mode=None)

#import seaborn as sns
#import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

import pickle

from importlib import reload
from sympy.physics.quantum.dagger import Dagger

In [3]:
# Local package imports
# Update this with setup & develop later
PROJECT_PATH = str(pathlib.Path().resolve().parent)
sys.path.append(PROJECT_PATH)

In [4]:
import hamiltonianlearner

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
import hamiltonianlearner.quantum_system_oracles.process_data as process_data
import hamiltonianlearner.quantum_system_oracles.simulate_nature as simulate_nature
import hamiltonianlearner.quantum_system_models.quantum_device_models as quantum_device_models
import hamiltonianlearner.learners.design_experiment as design_experiment

In [6]:
# estimators
import hamiltonianlearner.estimators.initial_estimators as initial_estimators
import hamiltonianlearner.estimators.mle_estimators as mle_estimators
import hamiltonianlearner.estimators.estimation_procedure as estimation_procedure

In [7]:
# For plotting purposes
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [8]:
# global configuration flags
from absl import app
from absl import flags
reload(flags)
FLAGS = flags.FLAGS

if "verbose" not in dir(FLAGS):
  flags.DEFINE_boolean('verbose', True, "generate verbose debugging output")
  flags.DEFINE_boolean('limit_search_space', False, "truncate MLE search space")
  
FLAGS(['verbose'])

['verbose']

In [9]:
import yaml
#import unit_tests

In [10]:
import functools

In [11]:
import hamiltonianlearner.utils.learner_experiment_utils as learner_experiment_utils
import hamiltonianlearner.utils.job_helper as job_helper

In [12]:
# Cool reloading
# Reference: https://stackoverflow.com/questions/28101895/reloading-packages-and-their-submodules-recursively-in-python

def reload_package(package):
    assert(hasattr(package, "__package__"))
    fn = package.__file__
    fn_dir = os.path.dirname(fn) + os.sep
    module_visit = {fn}
    del fn

    def reload_recursive_ex(module):
        reload(module)

        for module_child in vars(module).values():
            if isinstance(module_child, types.ModuleType):
                fn_child = getattr(module_child, "__file__", None)
                if (fn_child is not None) and fn_child.startswith(fn_dir):
                    if fn_child not in module_visit:
                        # print("reloading:", fn_child, "from", module)
                        module_visit.add(fn_child)
                        reload_recursive_ex(module_child)

    return reload_recursive_ex(package)

#### Global parameters/constants

In [13]:
def kron(a, b):
    return np.matrix(scipy.linalg.kron(a, b))

In [14]:
si = np.array([ [1, 0], [0, 1] ])
sx = np.array([ [0, 1], [1, 0] ])
sy = np.array([ [0, -1j], [1j, 0] ])
sz = np.array([ [1, 0], [0, -1] ])

# According to Ed's slides and answers verified -- match!
moset = {0: [si,scipy.linalg.expm(1j*(np.pi/4)*sy)], 1: [si,scipy.linalg.expm(-1j*(np.pi/4)*sx)], 2: [si,si]}
prepset = {0: [si, si], 1: [sx, si]}

In [15]:
# Time Stamps similar to data
time_stamps = np.linspace(1e-7,6e-7,81)

### Parameters of interest

In [16]:
# ibmq_boeblingen data

# Parameters of the different jobs
meas_level_expt = 1
n_shots = 512
n_job = 1
cr_amp_array = [0.24, 0.30, 0.36, 0.42, 0.48]

In [17]:
ind_amp = 1
pickle_result_filename = 'ibmq_boel_fixed_qs_data_aligned_A_0_%d_meas_%d_shots_%d_job_%d.pickle' % (100*cr_amp_array[ind_amp], meas_level_expt,
                                                                           n_shots, n_job)
pickle_result_file = 'Data/ibmq_boel/'+pickle_result_filename

## Large simulated dataset -- short time range

In [21]:
# For computing RMSE
xi_J_rmse = (10**6)*np.ones(6)

In [22]:
# Potential truth
J_truth = np.array([-4573872.71813216, -1456459.93269852,  -297217.75625596,
             6486501.41598311,  1397617.03924571,   406234.05359476])

xi_J = 10**np.amax(np.floor(np.log10(np.abs(J_truth))))*np.ones(len(J_truth))
xi_t = 1e-7

In [23]:
# Setup oracle -- simulator
print('Using J_num which we already discovered before!')

param_truth = quantum_device_models.transform_parameters(J_truth)

## Oracle properties
FLAG_simulator = True

## Noise Models
FLAG_readout_noise = True
FLAG_control_noise = True

# Control Noise
teff = quantum_device_models.data_driven_teff_noise_model(param_truth, FLAG_ibmq_boel=True)

misclassif_error = [0.0078125, 0.033203125]
expt_noise ={'readout':misclassif_error, 'imperfect_pulse_shaping':teff}

# Create oracle
env_qs = simulate_nature.Nature(J_truth, noise=expt_noise, expt_data=None,
                                FLAG_simulator=FLAG_simulator,
                                FLAG_readout_noise=FLAG_readout_noise,
                                FLAG_control_noise=FLAG_control_noise)

Using J_num which we already discovered before!
Simulator oracle setup


In [24]:
env_qs.print_info()

Oracle: Simulator
Noise Sources:
Readout Noise: FLAG=True, Value=[0.0078125, 0.033203125]
Control Noise: FLAG=True


In [25]:
# Create action space
time_stamps = ibm_data['time_stamps'][0:81]
time_stamps_nd = time_stamps/xi_t

A_cr = simulate_nature.Action_Space(moset, prepset, time_stamps_nd, xi_t, xi_J=xi_J)

In [26]:
# Sample data
np.random.seed(2021)
N_config = A_cr.N_actions
p_U = (1 / N_config) * np.ones(N_config)

N_queries = 255*N_config
X_p = A_cr.sample_action_space(env_qs, p_U, N_queries, FLAG_query=True)

In [27]:
len(X_p['samples'])

123930

In [28]:
A_cr.update_dict_action_space(X_p)

Baseline solver: FFT+Regression

In [29]:
loss_J_IC, J_num_IC = estimation_procedure.baseline_estimate(X_p, A_cr)

In [30]:
J_num_IC

array([-5669654.76474527, -3591498.01502238,  1750874.70079857,
        6460136.82314445,  -831873.95960098,  2569215.71693137])

MLE

In [31]:
loss_J, J_num = estimation_procedure.quick_mle_estimate(X_p, A_cr)

In [32]:
J_num

array([-4621066.7474512 , -1274349.81439786,  -124104.33153049,
        6547746.09894341,  1193407.08958774,   247607.04879776])

Summary

In [33]:
# Compare MLE losses
mle_est = mle_estimators.MLE_Estimator(X_p, xi_J)
print('Loss IC: %f' % mle_est.np_loss(J_num_IC, type_param='J'))
print('Loss MLE: %f' % mle_est.np_loss(J_num, type_param='J'))
print('Loss Potential truth: %f' % mle_est.np_loss(J_truth, type_param='J'))

Loss IC: 0.738251
Loss MLE: 0.509057
Loss Potential truth: 0.509276


In [34]:
J_mean_jobs = np.array([-4603928.82066976, -1336348.67108834,  -135657.6579083 ,
        6535572.99605797,  1261349.74890554,   258374.52928122])

In [35]:
mle_est.np_loss(J_mean_jobs, type_param='J')

0.509065432951569

In [39]:
estimation_procedure.normalized_L2_error(J_truth, J_mean_jobs, xi_J)

0.2902946488679517

In [40]:
estimation_procedure.normalized_L2_error(J_truth, J_num, xi_J)

0.3687475705575716

Above two errors is what I see approximately if I were to calculate RMSE from J_truth

In [41]:
estimation_procedure.normalized_L2_error(J_mean_jobs, J_num, xi_J)

0.0956628148272872

Above error is what I see in my empirical estimates